# scikit-learn 核心代码与操作全流程

```python
# 一、基础数据准备与加载
## 1. 数据格式与示例数据集（鸢尾花）
from sklearn.datasets import load_iris
import numpy as np

# 加载数据集
iris = load_iris()
X, y = iris.data, iris.target  # 特征矩阵 [n_samples, n_features] 与标签
print(f"数据集形状: {iris.data.shape}")  # (150, 4)
print(f"类别名称: {iris.target_names}")  # ['setosa' 'versicolor' 'virginica']


# 二、监督学习（Supervised Learning）
## 1. 数据集拆分（为模型训练做准备）
from sklearn.model_selection import train_test_split

# 划分训练集（75%）和测试集（25%）
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=0
)

## 2. K近邻分类器（KNeighborsClassifier）
from sklearn.neighbors import KNeighborsClassifier

# 初始化模型（1-近邻）
knn = KNeighborsClassifier(n_neighbors=1)
# 训练模型（所有Estimator均通过fit方法训练）
knn.fit(X_train, y_train)
# 预测测试集
predicted_knn = knn.predict(X_test)

## 3. 高斯朴素贝叶斯（GaussianNB）
from sklearn.naive_bayes import GaussianNB

# 初始化并训练模型
clf = GaussianNB()
clf.fit(X_train, y_train)
# 预测
predicted_gnb = clf.predict(X_test)

## 4. 分类性能评估
from sklearn import metrics

# K近邻分类评估
print("K近邻分类报告:")
print(metrics.classification_report(y_test, predicted_knn))
print("混淆矩阵:")
print(metrics.confusion_matrix(y_test, predicted_knn))

# 高斯朴素贝叶斯评估
print("\n高斯朴素贝叶斯分类报告:")
print(metrics.classification_report(y_test, predicted_gnb))

## 5. 回归问题（以线性回归为例）
from sklearn.linear_model import LinearRegression

# 假设使用鸢尾花数据的一个特征预测另一个特征（示例）
X_reg = X[:, 0].reshape(-1, 1)  # 取第一个特征作为输入
y_reg = X[:, 1]  # 取第二个特征作为目标
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.25, random_state=0
)

# 训练线性回归模型
model = LinearRegression()
model.fit(X_reg_train, y_reg_train)
predicted_reg = model.predict(X_reg_test)

# 回归性能评估（均方根误差）
rms = np.sqrt(np.mean((predicted_reg - y_reg_test) **2))
print(f"\n线性回归均方根误差: {rms:.4f}")

## 6. 梯度提升树回归
from sklearn.ensemble import GradientBoostingRegressor

# 训练梯度提升树模型
reg = GradientBoostingRegressor()
reg.fit(X_reg_train, y_reg_train)
predicted_gbr = reg.predict(X_reg_test)
gbr_rms = np.sqrt(np.mean((predicted_gbr - y_reg_test)** 2))
print(f"梯度提升树回归均方根误差: {gbr_rms:.4f}")


# 三、无监督学习（Unsupervised Learning）
## 1. 主成分分析（PCA）降维
from sklearn.decomposition import PCA

# 初始化PCA模型（降为2维）
pca = PCA(n_components=2)
# 拟合并转换数据（无监督学习常用fit_transform）
X_pca = pca.fit_transform(X)
print(f"\nPCA解释方差比例: {pca.explained_variance_ratio_}")  # 前两主成分的方差占比

## 2. t-SNE嵌入（高维数据可视化）
from sklearn.manifold import TSNE

# 初始化t-SNE模型
tsne = TSNE(n_components=2, random_state=0)
# t-SNE仅支持fit_transform，无法对新数据增量转换
X_2d = tsne.fit_transform(X)


# 四、模型优化与验证
## 1. 交叉验证（以朴素贝叶斯为例）
from sklearn.model_selection import cross_val_score

# 5折交叉验证
scores = cross_val_score(clf, X, y, cv=5)
print(f"\n交叉验证得分: {scores}")
print(f"平均得分: {np.mean(scores):.4f}")

## 2. 超参数优化（网格搜索）
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

# 定义参数网格（Ridge回归的正则化系数alpha）
param_grid = {'alpha': np.logspace(-3, -1, 30)}
# 初始化网格搜索（3折交叉验证）
gscv = GridSearchCV(Ridge(), param_grid, cv=3)
gscv.fit(X_train, y_train)
print(f"\n网格搜索最优参数: {gscv.best_params_}")

## 3. 内置交叉验证模型（RidgeCV）
from sklearn.linear_model import RidgeCV

# 自动选择最优alpha（内置交叉验证）
alphas = np.logspace(-3, 1, 10)
model_ridgecv = RidgeCV(alphas=alphas, cv=3)
model_ridgecv.fit(X_train, y_train)
print(f"RidgeCV最优正则化系数: {model_ridgecv.alpha_}")

## 4. 正则化效果示例（以多项式拟合为例）
# 假设X_poly为多项式特征，对比不同阶数的拟合效果
# 4阶多项式比9阶更简单，过拟合风险更低（正则化核心思想）


# 五、管道工具（串联特征工程与模型）
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

# 构建多项式特征+线性回归的管道
pipeline = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
# 管道直接调用fit和predict，内部自动处理特征转换
pipeline.fit(X_reg_train, y_reg_train)
pipeline_pred = pipeline.predict(X_reg_test)